In [1]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np

2024-07-17 10:55:57.369227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 10:55:57.490001: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 10:55:57.492610: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2024-07-17 10:55:57.492623: I tensorflow/compiler/xla/stream_ex

In [2]:
vocab_size = 5000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [3]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

25000 25000 25000 25000


In [4]:
word_idx = imdb.get_word_index()

word_idx = {i:word for word, i in word_idx.items()}

print([word_idx[i] for i in X_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [5]:
print("Max length of a review : ", len(max((X_train+X_test), key=len)))
print("Min length of a review : ", len(min((X_train+X_test), key=len)))

Max length of a review :  2697
Min length of a review :  70


In [6]:
from tensorflow.keras.preprocessing import sequence

# keeping a fixed length of all reviews to a max 400 words
max_words = 400

X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

X_valid, y_valid = X_train[:64], y_train[:64]
X_train_, y_train_ = X_train[64:], y_train[64:]


#### Simple RNN

In [7]:
# fixing every words embedding size to be 32
embd_len = 32

# Creating a RNN model
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(vocab_size, 
                        embd_len, 
                        input_length=max_words))

# In case of a stacked (more than one layer of RNN), use return_sequences=True
RNN_model.add(SimpleRNN(128, 
                        activation='tanh', 
                        return_sequences=False))

RNN_model.add(Dense(1, 
                    activation='sigmoid'))

# printing model summary
print(RNN_model.summary())

2024-07-17 10:56:02.325621: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-17 10:56:02.325649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MYTSL02451): /proc/driver/nvidia/version does not exist
2024-07-17 10:56:02.325966: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 32)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               20608     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 180,737
Trainable params: 180,737
Non-trainable params: 0
_________________________________________________________________
None


#### Parameters Calculation

1) Embedding Layer = vocab_size * embd_len = 5000*32 = 160000
2) Simple RNN Layer = (input_dims * units) + (units * units) + units = (32*128)+(128*128)+128 = 20608
3) Dense Layer = (input_dims * output_dims) + output_dims = (128*1)+1 = 129
4) Total params = 180,737

In [8]:
# compiling model
RNN_model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])

# Training the model
history = RNN_model.fit(X_train_,y_train_,
                        batch_size=64,
                        epochs=5,
                        verbose=1,
                        validation_data=(X_valid,y_valid))

# Printing model score on test data
print()
print("Simple_RNN Score ---> ", RNN_model.evaluate(X_test, y_test, verbose=0))

Epoch 1/5
390/390 [==============================] - 22s 54ms/step - loss: 0.6821 - accuracy: 0.5491 - val_loss: 0.6508 - val_accuracy: 0.6406
Epoch 2/5
390/390 [==============================] - 24s 62ms/step - loss: 0.5522 - accuracy: 0.7190 - val_loss: 0.4616 - val_accuracy: 0.7969
Epoch 3/5
390/390 [==============================] - 22s 57ms/step - loss: 0.4603 - accuracy: 0.7893 - val_loss: 0.4631 - val_accuracy: 0.8125
Epoch 4/5
390/390 [==============================] - 22s 57ms/step - loss: 0.4645 - accuracy: 0.7844 - val_loss: 0.6937 - val_accuracy: 0.5938
Epoch 5/5
390/390 [==============================] - 21s 55ms/step - loss: 0.4601 - accuracy: 0.7872 - val_loss: 0.5498 - val_accuracy: 0.7500

Simple_RNN Score --->  [0.4818555414676666, 0.7746800184249878]
